In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from quantfreedom.enums import CandleBodyType
from quantfreedom.indicators.tv_indicators import rma_tv
from plotly.subplots import make_subplots
from quantfreedom.indicators.tv_indicators import sma_tv

from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [ ]:
candles = dl_ex_candles(
    exchange="binance_us",
    symbol="BTCUSDT",
    timeframe="1h",
    candles_to_dl=400,
)

In [ ]:
candles_to_df(candles)

In [ ]:
open = candles[:, CandleBodyType.Close]
high = candles[:, CandleBodyType.High]
low = candles[:, CandleBodyType.Low]
close = candles[:, CandleBodyType.Close]

In [ ]:
lin_reg_candles = np.full_like(candles, np.nan)

In [ ]:
lin_reg_length = 11
x = np.arange(0, lin_reg_length)
A = np.vstack([x, np.ones(len(x))]).T
lin_reg_length_m_1 = lin_reg_length - 1

In [ ]:
for i in range(lin_reg_length_m_1, close.size):
    m, b = np.linalg.lstsq(A, open[i - lin_reg_length_m_1 : i + 1], rcond=None)[0]
    lin_reg_candles[i, CandleBodyType.Open] = b + m * lin_reg_length_m_1

    m, b = np.linalg.lstsq(A, high[i - lin_reg_length_m_1 : i + 1], rcond=None)[0]
    lin_reg_candles[i, CandleBodyType.High] = b + m * lin_reg_length_m_1

    m, b = np.linalg.lstsq(A, low[i - lin_reg_length_m_1 : i + 1], rcond=None)[0]
    lin_reg_candles[i, CandleBodyType.Low] = b + m * lin_reg_length_m_1

    m, b = np.linalg.lstsq(A, close[i - lin_reg_length_m_1 : i + 1], rcond=None)[0]
    lin_reg_candles[i, CandleBodyType.Close] = b + m * lin_reg_length_m_1

In [ ]:
smoothing_type = sma_tv
smoothing_length = 11

In [ ]:
signal = smoothing_type(
    source=lin_reg_candles[:, CandleBodyType.Close],
    length=smoothing_length,
)

In [ ]:
signal

In [ ]:
lin_reg_candles[:, CandleBodyType.Timestamp] = candles[:, CandleBodyType.Timestamp]
lin_reg_candles[:, CandleBodyType.Volume] = candles[:, CandleBodyType.Volume]